# Propensity based methods

In this notebook, we apply baseline propensity score based methods to
the previously extracted COVID-19 cohort. The goal is to estimate
the treatment effect of proning on oxygenation measured by P/F ratio.

## 0. Summary

Takeaways:
 - We still miss a lot of covariate data.
 - Proning is much more complicated, than I thought. It can be in fact
 a time varying treatment, but let's ignore that issue.
 - We can successfully model the difference of P/F ratio at inclusion and
 after 12-16h after proning started.
 - We use compare four techniques: outcome regression, weighting, blocking
 and propensity score matching.
 - No way to tell how good our predictions are.

 On the bright side:
 - All the relevant RCTs have $N <= 500$.
 - We have $N = 10000$.
 - There is an RCT that studied the average difference in the
 change of P/F ratio with (supine) $45$ and (prone) $63$, what gives $ATE = 18$.
 - Our estimates ranges roughly between 15-30.


#### To do and next steps:
- Extract more data
- Carefully build a propensity score model by: a. selecting covariates that
influences the outcome and the treatment assignment b. feature engineering
c. tuning the model with respect to some balance measure.
- Try matching on the covariates and more advanced matching
 packages (unfortunately in R)
- Try BART (unfortunately in R)
- Look at multivariate imbalance metrics implemented in PyTorch:
https://torch-two-sample.readthedocs.io/en/latest/#
- Compare all the methods.
- We may want to access whether ATE, ATT or ATC is our main interest.
- Wrap all the plotting and imbalance accessing functionalities into a new subpackage

Question:
 - Maybe we should change the methodology and take the proning as the first measurement after
 proning started if this is within one hour? Or one hour before, but do not
 forward-fill. We could trace back for sure that a prone position does not
 start as we have no heart rate measurement. There should be a window for each
 patient in which there is no heart rate measurement.
 
 Things to start from: only Guerin covariates, mention that if we include P/F ratio as 
 the outcome then we also need to include P/F ratio as the baseline but then a linear model will
 be misspecified as autoregression will do it. 

## 1. Loading the data

In [ ]:
import pandas as pd
import numpy as np

import sys, os

import seaborn as sns
import matplotlib.pyplot as plt

from causalinference import CausalModel

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

from scipy.stats import wasserstein_distance
from scipy import stats

We load data created by '01-create-use-case'. The data contains only sessions
fulfilling the inclusion criteria. The outcome is P/F ratio measured between
12h and 16h from the start of the proning.

In [ ]:
os.chdir('/home/adam/files/data/04012020/')
df = pd.read_csv('data_guerin_rct.csv')
df.start_timestamp = df.start_timestamp.astype('datetime64[ns]')
df.end_timestamp = df.end_timestamp.astype('datetime64[ns]')
df.info(max_cols=200)

We further process the data by dropping unnecessary observations and columns.

In [ ]:
print(df.has_died_during_session.value_counts())
df = df[~df.has_died_during_session]

In [ ]:
print("Before:")
print(df.treated.value_counts())
df.dropna(axis=0, how='any', subset=['pf_ratio_12h_outcome'], inplace=True)
print("After:")
print(df.treated.value_counts())

In [ ]:
df_plot = df
sns.distplot(df_plot['pf_ratio_inclusion_8h'],
             hist = True,
             kde = True,
             label='Inclusion')

sns.distplot(df_plot['pf_ratio_12h_outcome'],
             hist = True,
             kde = True,
             label='Outcome')
# Plot formatting
plt.legend(prop={'size': 12})
plt.title('P/F ratio improves for all patients')
plt.xlabel('pf_ratio')
plt.ylabel('Density')
plt.xlim(right=400)

plt.savefig('inclusion_8h_vs_outcome_12h.png')
# Figure comparing inclusion vs. outcome. Sessions included in the study with a
# non-missing outcome

In [ ]:
visualize_balance(df, 'treated', 'pf_ratio_12h_outcome')

In [ ]:
# Convert output to differences and see if this helps

df['pf_ratio_diff'] = df['pf_ratio_12h_outcome'] - df['pf_ratio_inclusion_8h']
df['pf_ratio_diff'].describe().round()

In [ ]:
visualize_balance(df, 'treated', 'pf_ratio_diff')

In [ ]:
df.info(max_cols=200)

In [ ]:
columns_to_drop_1 = df.iloc[:, 0:4].columns.tolist()
columns_to_drop_2 = df.iloc[:, 5:11].columns.tolist()
columns_to_drop_3 = df.iloc[:, 14:18].columns.tolist()
columns_to_drop = columns_to_drop_1 + columns_to_drop_2 + columns_to_drop_3
df_model = df.drop(columns=columns_to_drop)
df_model = df_model.drop(columns=['has_died_during_session', 'gender'])
# And old outcome
df_model = df_model.drop(columns=['pf_ratio_12h_outcome', 'fio2', 'po2'])
df_model.info()

In [ ]:
thresh = round(0.6 * len(df_model.index))
df_model = df_model.dropna(thresh=thresh, axis=1)

In [ ]:
df_model = df_model.drop(df_model.filter(regex='atc').columns, axis=1)
df_model = df_model.drop(df_model.filter(regex='nice').columns, axis=1)
df_model = df_model.drop(df_model.filter(regex='inclusion').columns, axis=1)

Finally, the shape of the data used for the purpose of the causal inference
is printed below.

In [ ]:
df_model.info()



## 2. Missing data imputation

In [ ]:
treated = df_model.iloc[:,0].values.astype('int')
t = df_model.loc[:, 'treated'].values
X = df_model.drop(columns=['treated', 'pf_ratio_diff']).values
y = df_model.loc[:, 'pf_ratio_diff'].values

In [ ]:
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
imp.fit(X)
X = imp.transform(X)

# Standardize the predictors
scaler = StandardScaler().fit(X)
X = scaler.transform(X)

## 3. Causal modelling

In [ ]:
# Instantiate CausalModel

causal = CausalModel(y, t, X)

### 3.1 Modelling methodology

We can divide the study into two phases:
1. #### Design phase
The goal of the design phase is to preprocess the data in order to ensure credible analysis.
E.g. by estimating the propensity score and balancing the covariates. Itterative back-and-forth
process. There is no standard way in the literature to do this.
2. #### Analysis phase
Once the data is prepared, treatment effects can safely be estimated.

#### Design phase: access initial balance

The variable with the biggest imbalance measured by the normalized differences
in average covariates:

$$\frac{\overline{X}_i^{t=1} - \overline{X}_i^{t=0}}{\sqrt{\frac{1}{2}(s_i^{t=1})^2 + (s_i^{t=0})^2}},$$

recommended by Imbens and Rubin (2015):

In [ ]:
print(causal.summary_stats)

In [ ]:
sorted_index_array = np.argsort(abs(causal.summary_stats['ndiff']))
sorted_array = causal.summary_stats['ndiff'][sorted_index_array]
rslt = sorted_array[-5 : ]
print(rslt)

idx = np.argpartition(abs(causal.summary_stats['ndiff']), -4)[-5:] + 1
imbalanced_covariates = df_model.columns[idx.tolist()]
print(imbalanced_covariates)
df_model[imbalanced_covariates].info()

In [ ]:
wass_dist = []
for _, column in enumerate(idx - 1):
    covariate_control = X[t][:, column]
    covariate_treated = X[~t][:, column]
    dist = wasserstein_distance(covariate_control,covariate_treated)
    wass_dist = wass_dist + [round(dist, 2)]

wass_dist

In [ ]:
# If p-value is low then we can reject the null hypothesis that
# the distributions of the two samples are the same
ks_test = []
for _, column in enumerate(idx - 1):
    covariate_control = X[t][:, column]
    covariate_treated = X[~t][:, column]
    test = stats.ks_2samp(covariate_control,covariate_treated)[1]
    ks_test = wass_dist + [round(test, 3)]

ks_test

Now we should look more closely into the imbalanced values

In [ ]:
df_model.corr()['pf_ratio_diff'].round(2)[1:]

In [ ]:
df_model[df_model.treated].corr()['pf_ratio_diff'].round(2)[1:]

In [ ]:
df_model[~df_model.treated].corr()['pf_ratio_diff'].round(2)[1:]

In [ ]:
for _, column in enumerate(imbalanced_covariates):
    visualize_balance(df_model, 'treated', column)


In [ ]:
# Hemoglobin is the most imbalanced

causal.est_propensity()
print(causal.propensity)

In [ ]:
sns.distplot(causal.raw_data['pscore'][t],
             hist = True,
             kde = True,
             label='Prone')

sns.distplot(causal.raw_data['pscore'][~t],
             hist = True,
             kde = True,
             label='Supine')

# Plot formatting
plt.legend(prop={'size': 12})
plt.title('Pscore')
plt.xlabel('pscore')
plt.ylabel('Density')



In [ ]:
SEED = 1234
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=SEED,
                         class_weight='balanced',
                         penalty='none').fit(X, t)
pscore = clf.predict_proba(X)[:, 1]
print(clf.coef_.round(2))

In [ ]:
sns.distplot(pscore[t],
             hist = True,
             kde = True,
             label='Prone')

sns.distplot(pscore[~t],
             hist = True,
             kde = True,
             label='Supine')

# Plot formatting
plt.legend(prop={'size': 12})
plt.title('Pscore')
plt.xlabel('pscore')
plt.ylabel('Density')

In [ ]:
# We assign the new p-score
causal.raw_data._dict['pscore'] = pscore

We trim samples to ensure positivity

In [ ]:
causal.trim_s()
print(causal.cutoff)
print(causal.summary_stats)

In [ ]:
causal.stratify_s()
print(causal.strata)


In [ ]:
for stratum in causal.strata:
    print(max(stratum.summary_stats['ndiff']))

Now we see that the imbalance decreased a little, but there is
still lot to do.

In [ ]:
#causal.reset()

#### Model


In [ ]:
causal.est_via_ols()
print(causal.estimates)

In [ ]:
for stratum in causal.strata:
    stratum.est_via_ols(adj=2)
[stratum.estimates['ols']['ate'] for stratum in causal.strata]

Taking the sample-weighted average of the above within-bin least squares estimates results in a propensity score
matching estimator that is commonly known as the blocking estimator.

In [ ]:
# sample-weighted average of the within-bin least squares estimates

causal.est_via_blocking()
print(causal.estimates)


In [ ]:
causal.est_via_matching(bias_adj=True)
print(causal.estimates)

In [ ]:
for stratum in causal.strata:
    stratum.est_via_matching()
[stratum.estimates['matching']['ate'] for stratum in causal.strata]


In [ ]:
causal.est_via_weighting()
print(causal.estimates)

In [ ]:
y = []
yerr = []
x_label = []

for method, result in dict(causal.estimates).items():
    y.append(result["ate"])
    yerr.append(result["ate_se"])
    x_label.append(method)

y.append(21.392)
yerr.append(0)
x_label.append("raw")

x = np.arange(len(y))

plt.errorbar(x=x, y=y, yerr=yerr, linestyle="none", capsize=5, marker="o")
plt.xticks(x, x_label)
plt.title("Estimated Effect Size", fontsize=18)
plt.hlines(y=18, xmin=-0.5, xmax = 4.5, linestyles="dashed")
#plt.xlim(-0.5,3.5);




In [ ]:
def visualize_balance(df, treated_column_name, covariate_column_name):

    df_plot = df[df[treated_column_name]]
    sns.distplot(df_plot[covariate_column_name],
                 hist = True,
                 kde = True,
                 label='Prone')
    xlim_diff = df_plot[covariate_column_name].quantile(q=0.98) - df_plot[covariate_column_name].quantile(q=0.96)
    xlim = df_plot[covariate_column_name].quantile(q=0.98) + 2*xlim_diff

    df_plot = df[~df[treated_column_name]]
    sns.distplot(df_plot[covariate_column_name],
                 hist = True,
                 kde = True,
                 label='Supine')
    xlim_diff = df_plot[covariate_column_name].quantile(q=0.98) - df_plot[covariate_column_name].quantile(q=0.96)
    xlim = max(df_plot[covariate_column_name].quantile(q=0.98) + 2*xlim_diff, xlim)
    # Plot formatting
    plt.legend(prop={'size': 12})
    plt.title('Distribution of {} in treated and control subpopulations.'.format(covariate_column_name))
    plt.xlabel(str(covariate_column_name))
    plt.ylabel('Density')
    plt.xlim(right=xlim)

    print("Mean value of {} in the treated subpopulation: {}.".format(
          covariate_column_name, round(df.loc[df[treated_column_name], covariate_column_name].mean(), 2)))
    print("Mean value of {} in the supine subpopulation: {}.".format(
          covariate_column_name, round(df.loc[~df[treated_column_name], covariate_column_name].mean()), 2))
    plt.show()

In [ ]:
df_plot = df_model[['peep',
                      'pressure_above_peep',
                      'tidal_volume',
                      'respiratory_rate_measured',
                      'respiratory_rate_measured_ventilator',
                      'lung_compliance_static',
                      'driving_pressure',
                      'peak_pressure']]
f = plt.figure(figsize=(19, 15))
plt.matshow(df_plot.corr(), fignum=f.number)
#plt.xticks(range(df_plot.shape[1]), df_model.columns, fontsize=14, rotation=45)
#plt.yticks(range(df_plot.shape[1]), df_model.columns, fontsize=14)
cb = plt.colorbar()
cb.ax.tick_params(labelsize=14)
plt.title('Correlation Matrix', fontsize=16);

In [ ]:
x = df_plot.columns[2]
y = df_plot.columns[5]

sns.scatterplot(data=df_plot, x=x, y=y)

In [ ]:
with sns.axes_style('white'):
    sns.jointplot(y=y,
                  x=x,
                  data=df_plot,
                  kind='kde')
#plt.xlim(left=0, right=40)
#plt.ylim(bottom=0, top=50)
plt.show()

In [ ]:
%reset

In [ ]:

import numpy as np
from sklearn.impute import KNNImputer
nan = np.nan
X = [[1, 2, nan], [3, 4, 3], [nan, 6, 5], [8, 8, 7]]
imputer = KNNImputer(n_neighbors=2, weights="uniform")
imputer.fit_transform(X)

In [ ]:
SEED = 1234
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=SEED,
                         class_weight='balanced',
                         penalty='none').fit(X[:, 23:30], t)
pscore = clf.predict_proba(X[:, 23:30])[:, 1]
print(clf.coef_.round(2))

In [ ]:
sns.distplot(pscore[t],
             hist = True,
             kde = True,
             label='Prone')

sns.distplot(pscore[~t],
             hist = True,
             kde = True,
             label='Supine')

# Plot formatting
plt.legend(prop={'size': 12})
plt.title('Pscore')
plt.xlabel('pscore')
plt.ylabel('Density')